In [4]:
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.llms import CTransformers

In [22]:
!pip install --upgrade langchain
!pip install faiss-cpu

   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/14.5 MB 1.2 MB/s eta 0:00:13
   -- ------------------------------------- 0.8/14.5 MB 7.2 MB/s eta 0:00:02
   ----- ---------------------------------- 2.0/14.5 MB 12.6 MB/s eta 0:00:01
   --------- ------------------------------ 3.3/14.5 MB 16.2 MB/s eta 0:00:01
   ------------ --------------------------- 4.5/14.5 MB 17.9 MB/s eta 0:00:01
   --------------- ------------------------ 5.7/14.5 MB 19.3 MB/s eta 0:00:01
   ---------------- ----------------------- 6.1/14.5 MB 18.5 MB/s eta 0:00:01
   -------------------- ------------------- 7.3/14.5 MB 19.4 MB/s eta 0:00:01
   ----------------------- ---------------- 8.4/14.5 MB 20.0 MB/s eta 0:00:01
   ------------------------- -------------- 9.4/14.5 MB 20.1 MB/s eta 0:00:01
   ---------------------------- ----------- 10.4/14.5 MB 23.4 MB/s eta 0:00:01
   

In [10]:
DEFAULT_PROMPT="You are a book summarizer and can provide updated summaries"
B_INST, E_INST="[INST]", "[/INST]"
B_SYS,E_SYS="<<SYS>>>\n", "\n<<<SYS>>>\n\n"
instruction="Convert the following text from english to french \n\n {text}"

instruction="Give a proper summary of the book of \n\n {text}"
SYSTEM_PROMPT=B_SYS+DEFAULT_PROMPT+E_SYS
template=B_INST+SYSTEM_PROMPT+instruction+E_INST

In [11]:
template

'[INST]<<SYS>>>\nYou are a book summarizer and can provide updated summaries\n<<<SYS>>>\n\nGive a proper summary of the book of \n\n {text}[/INST]'

In [12]:
prompt=PromptTemplate(template=template, input_variables=['text'])

In [13]:
llm=CTransformers(model='llama-2-7b-chat.ggmlv3.q4_0.bin',
                  model_type='llama',
                  config={'max_new_tokens': 128,'temperature':0.01})
llmchain=LLMChain(llm=llm, prompt=prompt)

In [14]:
print(llmchain.run("Harry Potter"))

  Certainly! Here is an updated summary of the book "Harry Potter" by J.K. Rowling:

"Harry Potter" is a magical adventure that follows the journey of a young wizard named Harry Potter as he attends Hogwarts School of Witchcraft and Wizardry. Born to a poor but loving family, Harry discovers on his eleventh birthday that he is the son of two powerful wizards who were killed by the dark wizard Lord Voldemort.

Harry begins his first year at Hogwarts, where


In [28]:
from  langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers
from langchain import PromptTemplate


In [17]:
data_pdf=PyPDFDirectoryLoader('pdfs')
loaded=data_pdf.load()
splitter=RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
split_chunks=splitter.split_documents(loaded)

In [19]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                 model_kwargs={'device':'cpu'})

c:\Users\satya\anaconda3\envs\transformers\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
modules.json: 100%|██████████| 349/349 [00:00<00:00, 116kB/s]
c:\Users\satya\anaconda3\envs\transformers\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\satya\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space 

In [36]:
vector_store=FAISS.from_documents(split_chunks, embeddings)

In [31]:
llm=CTransformers(model='llama-2-7b-chat.ggmlv3.q4_0.bin',
                   model_type='llama',
                   config={'max_new_tokens': 128,'temperature':0.01})

In [93]:
DEFAULT_PROMPT="You are connected to a RAG system please use that to answer questions"

B_INST, E_INST="[INST]", "[/INST]"
B_SYS,E_SYS="<<SYS>>>\n", "\n<<<SYS>>>\n\n"
instruction="Please anwer questions {question} based on context {context} specific to the research paper "

SYSTEM_PROMPT=B_SYS+DEFAULT_PROMPT+E_SYS
template=B_INST+SYSTEM_PROMPT+instruction+E_INST

In [87]:
template

'[INST]<<SYS>>>\nYou are connected to a RAG system please use that to answer questions\n<<<SYS>>>\n\nPlease anwer questions {question} based on context {context} specific to the research paper [/INST]'

In [88]:
qa_prompt=PromptTemplate(template=template,input_variables=['context', 'question'])

In [66]:
DEFAULT_PROMPT="Use the following pieces of information to answer the users question Context: {context} Question: {question} Answer in brief an objectively"

B_INST, E_INST="[INST]", "[/INST]"
B_SYS,E_SYS="<<SYS>>>\n", "\n<<<SYS>>>\n\n"
instruction="Can you tell me about the dataset used in the paper? "

SYSTEM_PROMPT=B_SYS+DEFAULT_PROMPT+E_SYS
template=B_INST+SYSTEM_PROMPT+instruction+E_INST

In [67]:
DEFAULT_PROMPT="Use the following pieces of information to answer the users question Context: {context} Question: {question}"


In [94]:
qa_prompt=PromptTemplate(template=template,input_variables=['context', 'question'])

In [95]:
chain=RetrievalQA.from_chain_type(llm=llm,
                                  chain_type='stuff',
                                  retriever=vector_store.as_retriever(search_kwargs={'k':2}),
                                  return_source_documents=True,
                                  chain_type_kwargs={'prompt':qa_prompt})

In [100]:
user_input='what is the performance of the model on fall detection the author developed relating to Siamese networks?'
result=chain({'query':user_input})

In [101]:
print(result)

{'query': 'what is the performance of the model on fall detection the author developed relating to Siamese networks?', 'result': "  RAG System:\n\nR - Recall: The model achieved an accuracy of 95% in detecting falls.\nA - Precision: The model had a precision of 90% in identifying non-falls.\nG - F1 Score: The model's F1 score for fall detection was 92%.\n\nBased on the context, Siamese networks are indeed effective in fall detection tasks. The novel network architecture developed in the research paper leverages the few examples available to learn and improve its performance over time. The use of Siamese networks allows the model", 'source_documents': [Document(page_content='is paramount for fall detection.\nSiamese networks are designed to learn from few examples', metadata={'source': 'pdfs\\IEEE_SENSORS_LETTERS.pdf', 'page': 0}), Document(page_content='A fall detection system (FEDS) using a novel Siamese network', metadata={'source': 'pdfs\\IEEE_SENSORS_LETTERS.pdf', 'page': 3})]}
